In [1]:
import yfinance as yf
import pandas as pd

In [2]:
# 凍傷のエクセルを読み込み
df = pd.read_excel('data/toushou/data_j.xls')
# print(df)

# 構成区分を一覧に出してみる
# kibo_group = df.groupby('規模区分')
# print(kibo_group.count())

# TOPIX Core30 に絞り込み
core30 = df.query('規模区分 == "TOPIX Core30"').rename(columns={'コード':'code'})
# print(core30)
# コードの一覧を出力する
core30.to_csv('data/toushou/code_core30.csv', columns=['code'], index=False)

In [3]:
# core30をyfinance用に文字列加工する
data = pd.read_csv('data/toushou/code_core30.csv')
# print(data)
stocks = [str(s)+".T" for s in data.code]
stocks.append("^N225")
tickers = yf.Tickers(" ".join(stocks))

closes   = [] # 終値

for i in range(len(tickers.tickers)):
    closes.append(tickers.tickers[i].history(period="max").Close)

closes = pd.DataFrame(closes).T   # DataFrame化
closes.columns = stocks           # カラム名の設定
closes = closes.ffill()           # 欠損データの補完

print(closes)

ConnectionError: HTTPSConnectionPool(host='query1.finance.yahoo.com', port=443): Max retries exceeded with url: /v8/finance/chart/8001.T?period1=-2208988800&period2=1609081559&interval=1d&includePrePost=False&events=div%2Csplits (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x11c92cdd0>: Failed to establish a new connection: [Errno 65] No route to host'))

In [4]:
# とりあえず日立製作所を表示してみた
closes['6501.T'].plot()

TypeError: list indices must be integers or slices, not str

In [5]:
# 日立製作所に限って分析していってみよう
ticker = yf.Ticker('6501.T')


In [6]:
# 当期純利益
earning = ticker.financials.T["Net Income"]
print(earning)



2020-03-31     87596000000.0
2019-03-31    222546000000.0
2018-03-31    362988000000.0
2017-03-31    231261000000.0
Name: Net Income, dtype: object


In [7]:
# 自己資本
equity = ticker.balance_sheet.T["Total Stockholder Equity"]
print(equity)


2020-03-31    3.159986e+12
2019-03-31    3.262603e+12
2018-03-31    3.278024e+12
2017-03-31    2.967085e+12
Name: Total Stockholder Equity, dtype: float64


In [8]:
# 発行株数
share = ticker.info["sharesOutstanding"]
print(share)

966835008


In [9]:
eps = earning/share # EPS
roe = earning/equity # ROE
print(eps)
print(roe)


2020-03-31     90.600774
2019-03-31    230.179915
2018-03-31    375.439446
2017-03-31    239.193863
Name: Net Income, dtype: object

2020-03-31    0.027720
2019-03-31    0.068211
2018-03-31    0.110734
2017-03-31    0.077942
dtype: object
